In [41]:
#!/usr/bin/python3

# Import statements
import cv2
import math
import numpy as np


def main():
    
    cap = cv2.VideoCapture('test_videos/challenge.mp4')
    
    ret = True
    
    while(ret):
        ret, frame = cap.read()
        if(ret != False):
            processImage(frame)
    
    cv2.destroyAllWindows()
    
    
def processImage(imgSource):
    
    # Convert image to grayscale
    imgGray = cv2.cvtColor(imgSource, cv2.COLOR_BGR2GRAY)

    # Smoothen image
    imgBlur = cv2.GaussianBlur(imgGray, (3,3), 1)

    # Get edge image using Canny edge detector
    threshLow = 0
    threshHigh = 150
    imgEdge = cv2.Canny(imgBlur, threshLow, threshHigh)

    # Create mask to select only the lane in front of vehicle
    mask = np.copy(imgEdge) * 0
    vertices = np.array([[(200, 663), (590, 450), (735, 450), (1140, 663)]])
    cv2.fillPoly(mask, vertices, (255, 255, 255))
        
    # Apply mask to the imag
    imgMasked = cv2.bitwise_and(imgEdge, mask)

    # Draw lane lines
    drawLines(imgMasked, imgSource)
    
    
def drawLines(imgMasked, imgSource):
    
    # Get lines from masked edge image using Hough transform
    rho = 1
    theta = np.pi/180
    threshold = 15
    minLineLength = 100
    maxLineGap = 100
    lines = cv2.HoughLinesP(imgMasked, rho, theta, threshold, np.array([]), minLineLength, maxLineGap)
    
    if not(lines is None):
        slopePos = []; slopeNeg = []; ptsLeft = []; ptsRight = []

        # Calcualte slope
        for line in lines:
            for x1, y1, x2, y2 in line:
                if((x2-x1)!=0 and (y2-y1)!=0):
                    slope = float(y2-y1)/float(x2-x1)
                    angle = np.rad2deg(np.arctan2(y2-y1, x2-x1))
                    
                    if((angle > 30 and angle < 40) or (angle < -30 and angle > -40)):
                        # Right Line
                        if(slope < 0):
                            slopeNeg.append(slope)
                            ptsRight.append([x1, y1])
                        # Left Line
                        else:
                            slopePos.append(slope)
                            ptsLeft.append([x1,y1])
        
        if(len(slopeNeg)!=0 and len(slopePos)!=0):
            #slopeRight = np.mean(slopeNeg)
            #slopeLeft = np.mean(slopePos)
            slopeRight = slopeNeg[0]
            slopeLeft = slopePos[0]

            # Create black image of same size as edge image
            imgLines = np.copy(imgSource) * 0

            yTop = 460

            # Draw Right Line
            intercept = ptsRight[0][1] - slopeRight * ptsRight[0][0]
            xBottom = int( (imgSource.shape[0] - intercept) / slopeRight )
            xTop = int( (yTop - intercept) / slopeRight )
            cv2.line(imgLines, (xTop, yTop), (xBottom, imgSource.shape[0]), (200,50,0), 15)

            # Draw Left Line
            intercept = ptsLeft[0][1] - slopeLeft * ptsLeft[0][0]
            xBottom = int( (imgSource.shape[0] - intercept) / slopeLeft )
            xTop = int( (yTop - intercept) / slopeLeft )
            cv2.line(imgLines, (xTop, yTop), (xBottom, imgSource.shape[0]), (200,50,0), 15)

            # Add to source image
            imgLines = cv2.addWeighted(imgSource, 1.0, imgLines, 0.8, 1.0)

            # Display annotated image
            cv2.imshow('Image Lines', imgLines)
        
        else:
            cv2.imshow('Image Lines', imgSource)
            
    else:
        cv2.imshow('Image Lines', imgSource)
    
    key = cv2.waitKey(23)


if __name__ == "__main__": 
    main()